# Cantilever Beam: Finite Element Method (FEM)
This tutorial demonstrates how to simulate a cantilever beam using the **Finite Element Method (FEM)** in SOFA. You will learn how to define a 3D grid topology, apply fixed constraints using a region of interest (ROI), and model elastic deformation with hexahedral elements.

### Learning Objectives
- Define a 3D structured grid using `RegularGridTopology`.
- Understand the role of `MechanicalObject` and `MeshMatrixMass` in 3D simulations.
- Use `BoxROI` to select indices for constraints.
- Compare **StaticSolver** (equilibrium state) vs. **Dynamic Solver** (oscillating motion).
- Configure ODE solvers like `BDFOdeSolver` for stable time integration.
- Model linear elasticity using `HexahedronFEMForceField`.
- Visualize 3D mesh results and animations using `pyvista`.

---

## 1. Environment Setup
We begin by importing the core SOFA modules and initializing the runtime. We also import `numpy` and `pyvista` for data manipulation and visualization.

In [ ]:
import Sofa
import SofaRuntime
import numpy as np
import pyvista as pv
SofaRuntime.init()

## 2. Creating the Scene Graph
We create the `root` node and define gravity. In this simulation, we use a standard gravity vector along the negative Y-axis.

In [ ]:
root = Sofa.Core.Node("root")
root.gravity.value = [0.0, -9.81, 0.0]
root.dt.value = 0.01 # Simulation time step

## 3. Simulation Components

### Animation Loop and Solvers
In SOFA, the simulation is orchestrated by an **Animation Loop**. For this first part, we use a `StaticSolver`. 

Unlike dynamic solvers that integrate motion over time, a **static solver** finds the equilibrium state where internal elastic forces balance external forces (like gravity). This is equivalent to finding the state where the velocity and acceleration of all particles are zero.

We also use:
- **NewtonRaphsonSolver**: A non-linear solver used to find the equilibrium by iteratively minimizing the residual forces.
- **CGLinearSolver**: A Conjugate Gradient linear solver to solve the underlying system of equations at each Newton iteration.

In [ ]:
root.addObject("DefaultAnimationLoop")

# ODE / Static Solvers
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Backward")
root.addObject("NewtonRaphsonSolver", 
               maxNbIterationsNewton=100, 
               maxNbIterationsLineSearch=10,
               warnWhenLineSearchFails=False,
               warnWhenDiverge=False)

root.addObject("StaticSolver", name="solver")

# Linear Solver
SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Iterative")
root.addObject("CGLinearSolver", iterations=25, tolerance=1e-5, threshold=1e-5)

## 4. Topology and State
A cantilever beam requires a geometric representation. We use a **structured grid** of hexahedral elements.

### Regular Grid Topology
`RegularGridTopology` creates a box-shaped grid defined by the number of samples (`nx`, `ny`, `nz`) and its spatial bounds.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Topology.Container.Grid")

# Define a beam elongated along the Z-axis
grid = root.addObject('RegularGridTopology', name="grid", 
                    nx=4, ny=4, nz=20, 
                    xmin=-9.0, xmax=-6.0, 
                    ymin=0.0, ymax=3.0, 
                    zmin=0.0, zmax=19.0)

### Mechanical State
The `MechanicalObject` stores the positions of the vertices (DOFs).

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
state = root.addObject("MechanicalObject", template="Vec3", name="state")

### Mass
`MeshMatrixMass` computes the mass matrix based on the density and the volume of the elements. Here, we specify a `totalMass`.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("MeshMatrixMass", template="Vec3,Vec3", name="mass", totalMass=320.0)

## 5. Constraints and Forces

### Fixed Constraint (BoxROI)
To create a *cantilever* beam, one end must be fixed. We use `BoxROI` (Region of Interest) to select all vertices within a specific volume, then pass those indices to a `FixedProjectiveConstraint`.

In [ ]:
# 1. Select the base of the beam (z near 0)
SofaRuntime.importPlugin("Sofa.Component.Engine.Select")
box = root.addObject('BoxROI', name="box", 
                    box=[-10.0, -1.0, -0.1,  -5.0, 4.0, 0.1], 
                    drawBoxes=True)

# 2. Fix the selected indices
SofaRuntime.importPlugin("Sofa.Component.Constraint.Projective")
root.addObject('FixedProjectiveConstraint', indices="@box.indices")

### Linear Elasticity (FEM)
The `HexahedronFEMForceField` computes the internal elastic forces of the beam based on the Finite Element Method.
- **Young Modulus**: Represents the stiffness of the material.
- **Poisson Ratio**: Represents the expansion/contraction of the material in directions perpendicular to the direction of compression.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.SolidMechanics.FEM.Elastic")
root.addObject('HexahedronFEMForceField', name="FEM", 
               youngModulus=20000.0, 
               poissonRatio=0.3, 
               method="large")

## 6. Running the Simulation
We initialize the scene and perform one or more steps. Since we use a `StaticSolver`, the simulation will attempt to reach the equilibrium position (the bent state) immediately.

In [ ]:
# Initialize the scene graph
Sofa.Simulation.initRoot(root)

# Capture initial (undeformed) positions
initial_positions = np.array(state.position.value, dtype=np.float32).copy()

# Perform one static step to reach equilibrium
Sofa.Simulation.animate(root, root.dt.value)

# Capture final (deformed) positions
final_positions = np.array(state.position.value, dtype=np.float32).copy()

print("Simulation complete.")

## 7. Static Visualization
We use the `pyvista` library to visualize the 3D mesh of the cantilever beam at its equilibrium state.

In [ ]:
# Create the beam mesh
# grid.triangles.value provides the connectivity for the visualization
# pyvista expects cells in the format [n, id1, id2, id3, ...]
triangles = np.array(grid.triangles.value)
cells = np.hstack(np.c_[np.full(len(triangles), 3), triangles])
mesh = pv.PolyData(state.position.value, cells)

# Display the mesh
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(mesh, color="orange", opacity=1.0, show_edges=True)
plotter.camera_position = [(25.0, -5.0, 9.5), (-7.5, -5.0, 9.5), (0, 1, 0)]
plotter.add_axes()
# Use 'static' backend if trame kills the kernel
# To try interactive rendering again, switch to 'client' or 'html'
plotter.show(jupyter_backend='client')

## 8. Dynamic 3D Visualization (pyvista)
In the previous sections, we used a `StaticSolver` to find the final resting state (equilibrium) of the beam under gravity. In this section, we transition to a **dynamic simulation** to observe how the beam oscillates before stabilizing.

### Key Components for Dynamics
- **Time Integration (`BDFOdeSolver`)**: To simulate motion over time, we need an ODE solver. The **Backward Differentiation Formula (BDF)** is an implicit, multi-step method known for its stability in stiff systems (like FEM). We use `order=2` for second-order accuracy.
- **Direct Linear Solver (`SparseLDLSolver`)**: While `CGLinearSolver` is efficient for static cases, dynamic FEM simulations often benefit from **direct solvers** like `SparseLDLSolver`. These solvers factorize the system matrix, providing more robust results when the geometry undergoes significant motion.
- **Animation Loop (`DefaultAnimationLoop`)**: This component orchestrates the simulation steps, calling the solvers at each time increment (`dt`).

We will compute **100 time steps** and store the vertex positions at each step to create an interactive animation.

In [ ]:
root_dynamic = Sofa.Core.Node("root")
root_dynamic.dt.value = 0.1
root_dynamic.addObject("DefaultAnimationLoop")
root_dynamic.addObject("NewtonRaphsonSolver", 
               maxNbIterationsNewton=1, 
               maxNbIterationsLineSearch=10,
               warnWhenLineSearchFails=False,
               warnWhenDiverge=False)
root_dynamic.addObject("BDFOdeSolver", name="solver", order=2)

SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Direct")
root_dynamic.addObject("SparseLDLSolver", template="CompressedRowSparseMatrixMat3x3")

root_dynamic.addObject('RegularGridTopology', name="grid", 
                    nx=4, ny=4, nz=20, 
                    xmin=-9.0, xmax=-6.0, 
                    ymin=0.0, ymax=3.0, 
                    zmin=0.0, zmax=19.0)
state_dynamic = root_dynamic.addObject("MechanicalObject", template="Vec3", name="state")
root_dynamic.addObject("MeshMatrixMass", template="Vec3,Vec3", name="mass", totalMass=320.0)

box = root_dynamic.addObject('BoxROI', name="box", 
                    box=[-10.0, -1.0, -0.1,  -5.0, 4.0, 0.1], 
                    drawBoxes=True)
root_dynamic.addObject('FixedProjectiveConstraint', indices="@box.indices")

root_dynamic.addObject('HexahedronFEMForceField', name="FEM", 
               youngModulus=20000.0, 
               poissonRatio=0.3, 
               method="large")

Sofa.Simulation.initRoot(root_dynamic)

# Below, we store the results of 100 time steps for animation.
vertices_frame = [np.array(state_dynamic.position.value, dtype=np.float32).copy()]

for i in range(100):
    Sofa.Simulation.animate(root_dynamic, root_dynamic.dt.value)
    vertices_frame.append(np.array(state_dynamic.position.value, dtype=np.float32).copy())

print(f"Simulation complete: {len(vertices_frame)} frames captured.")

In [ ]:
# Create the initial mesh
triangles = np.array(grid.triangles.value)
cells = np.hstack(np.c_[np.full(len(triangles), 3), triangles])
mesh_anim = pv.PolyData(vertices_frame[0], cells)

# Animation parameters
frames = len(vertices_frame)

# Create a plotter for animation
plotter = pv.Plotter(notebook=True, window_size=(600, 400))
plotter.enable_anti_aliasing()
plotter.add_mesh(mesh_anim, color="blue", opacity=1.0, show_edges=True)
plotter.camera_position = [(35.0, -2.0, 9.5), (-7.5, -2.0, 9.5), (0, 1, 0)]
plotter.add_axes()

# Callback for the slider
def update_frame(value):
    mesh_anim.points = vertices_frame[int(value)]

# Add a slider widget to control the frame
plotter.add_slider_widget(update_frame, [0, frames - 1], title='Frame', value=0)

# Display the plot
plotter.show(jupyter_backend='client')

### Summary and Analysis
In this tutorial, we simulated a 3D cantilever beam using the Finite Element Method (FEM). 

#### Key takeaways:
- **Topology**: The `RegularGridTopology` allows for efficient creation of structured hexahedral meshes.
- **Static vs. Dynamic**: 
    - The **StaticSolver** finds the final equilibrium state (resting position) by balancing forces directly.
    - The **BDFOdeSolver** (Dynamic) integrates the equations of motion over time, allowing us to see the oscillations and the path towards equilibrium.
- **Physical Modeling**: The `HexahedronFEMForceField` captures the material properties through the Young's Modulus and Poisson's Ratio.
- **Visualization**: Using `pyvista`, we can visualize both static deformation and time-varying animations directly in the notebook.

The visualization shows the beam bending downwards due to gravity, with the fixed end (ROI) correctly maintaining its position while the free end exhibits the most significant displacement. In the dynamic version, we observe the characteristic damping and oscillation of a stiff elastic structure.